In [1]:
import io
import os
import boto3
import numpy as np
import pandas as pd

In [2]:
aws_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret = os.environ['AWS_SECRET_ACCESS_KEY']
client = boto3.client('s3'
                    ,aws_access_key_id=aws_id
                    ,aws_secret_access_key=aws_secret)

train_obj = client.get_object(Bucket='food-delivery-churn', Key=f"churn_45_train.csv")
test_obj = client.get_object(Bucket='food-delivery-churn', Key=f"churn_45_test.csv")

X_train = pd.read_csv(io.BytesIO(train_obj['Body'].read()), encoding='utf8')
X_train = X_train.drop(['user_id', 'signup_time_utc', 'last_order_time_utc'], axis=1)
y_train = X_train.pop('churned_user')

X_test = pd.read_csv(io.BytesIO(test_obj['Body'].read()), encoding='utf8')
X_test = X_test.drop(['user_id', 'signup_time_utc', 'last_order_time_utc'], axis=1)
y_test = X_test.pop('churned_user')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
X_train.select_dtypes(include=['object']).columns.tolist()

['city_name',
 'city_group',
 'acquisition_category',
 'acquisition_subcategory',
 'language',
 'first_order_delivered_on_time']

In [30]:
X_train['first_order_delivered_on_time'].unique()

array([True, False, nan], dtype=object)

In [31]:
X_train.head(15)

,city_name,city_group,foreign_user,acquisition_category,acquisition_subcategory,language,signup_to_order_hours,days_since_signup,first_order_driver_rating,first_order_avg_meal_rating,...,first_order_payment,first_order_discount_amount,first_order_discount_percent,first_order_meal_reviews,first_30_day_orders,first_30_day_avg_meal_rating,first_30_day_avg_driver_rating,first_30_day_avg_gmv,first_30_day_discount_percent,first_30_day_subscription_user
0,Pentos,City,False,OwnedChannel,InApp,unknown,1.066667,53,-1,-1.0,...,66.950000,4.985200,0.069301,-1,1,-1.0,-1.0,71.935200,0.069301,False
1,Sathar,City,False,OwnedChannel,InApp,domestic,0.833333,59,-1,-1.0,...,46.400000,3.989500,0.079173,-1,1,-1.0,-1.0,50.389500,0.079173,False
2,Oldtown,College Town,False,Referral,Referral,domestic,23.216667,41,-1,5.0,...,18.310000,10.000000,0.353232,2,5,5.0,-1.0,24.051700,0.157648,False
3,Tolos,City,True,Others,Others,foreign,2259.633333,401,-1,-1.0,...,83.210000,2.990000,0.034687,-1,1,-1.0,-1.0,86.200000,0.034687,False
4,Volantis,City,True,Referral,Referral,domestic,0.683333,64,-1,-1.0,...,2.160306,7.633972,0.779432,-1,1,-1.0,-1.0,9.794278,0.779432,False
5,Old Valyria,City,True,Others,Others,foreign,140.216667,65,-1,-1.0,...,223.820000,0.004400,0.000020,-1,1,-1.0,-1.0,223.824400,0.000020,False
6,Old Valyria,City,True,Others,Others,foreign,2851.850000,212,-1,-1.0,...,56.210000,0.003600,0.000064,-1,1,-1.0,-1.0,56.213600,0.000064,False
7,Volantis,City,True,Others,Others,foreign,527.100000,255,-1,2.5,...,20.450388,6.312979,0.235881,4,3,2.5,-1.0,27.890593,0.244321,False
8,Volantis,City,False,DigitalAds,FacebookAds,domestic,0.600000,159,-1,3.0,...,0.763359,7.000002,0.901672,1,3,4.0,-1.0,7.258806,0.700636,False
9,Samyrian,College Town,True,Others,Others,foreign,5043.233333,361,-1,2.0,...,36.130000,4.692300,0.114945,4,2,2.0,-1.0,50.914900,0.046055,False


In [39]:
def convert_cat_to_int(X_train, X_test):
        """ Converts string objects in categorical data to integers to use for training models.
        Args:
            None

        Returns:
            None
            Converts categorical object columns in self.X_train to categorical integer columns.
        """
        full_data = pd.concat([X_train, X_test], axis=0)
        object_cols = full_data.select_dtypes(include=['object']).columns.tolist()

        for col in object_cols:
            col_dict = {}

            for idx, category in enumerate(full_data[col].unique()):
                col_dict[category] = idx
            
            X_train[col] = X_train[col].map(lambda x: col_dict[x])
            X_test[col] = X_test[col].map(lambda x: col_dict[x])

In [40]:
convert_cat_to_int(X_train, X_test)

In [38]:
full_data = pd.concat([X_train, X_test], axis=0)

,city_name,city_group,foreign_user,acquisition_category,acquisition_subcategory,language,signup_to_order_hours,days_since_signup,first_order_driver_rating,first_order_avg_meal_rating,...,first_order_payment,first_order_discount_amount,first_order_discount_percent,first_order_meal_reviews,first_30_day_orders,first_30_day_avg_meal_rating,first_30_day_avg_driver_rating,first_30_day_avg_gmv,first_30_day_discount_percent,first_30_day_subscription_user
0,Pentos,City,False,OwnedChannel,InApp,unknown,1.066667,53,-1,-1.0,...,66.950000,4.985200,0.069301,-1,1,-1.0,-1.0,71.935200,0.069301,False
1,Sathar,City,False,OwnedChannel,InApp,domestic,0.833333,59,-1,-1.0,...,46.400000,3.989500,0.079173,-1,1,-1.0,-1.0,50.389500,0.079173,False
2,Oldtown,College Town,False,Referral,Referral,domestic,23.216667,41,-1,5.0,...,18.310000,10.000000,0.353232,2,5,5.0,-1.0,24.051700,0.157648,False
3,Tolos,City,True,Others,Others,foreign,2259.633333,401,-1,-1.0,...,83.210000,2.990000,0.034687,-1,1,-1.0,-1.0,86.200000,0.034687,False
4,Volantis,City,True,Referral,Referral,domestic,0.683333,64,-1,-1.0,...,2.160306,7.633972,0.779432,-1,1,-1.0,-1.0,9.794278,0.779432,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35276,Ghozi,City,False,Others,Others,domestic,2703.683333,325,-1,-1.0,...,26.690000,3.990000,0.130052,-1,3,-1.0,-1.0,36.968133,0.153054,False
35277,Qarth,City,True,OwnedChannel,InApp,domestic,112.083333,89,-1,-1.0,...,14.040000,4.986500,0.262082,-1,2,-1.0,-1.0,40.593250,0.061420,False
35278,Pentos,City,False,OwnedChannel,InApp,unknown,2187.066667,119,-1,5.0,...,43.610000,3.990000,0.083824,1,1,5.0,-1.0,47.600000,0.083824,False
35279,Volantis,City,False,OwnedChannel,InApp,domestic,0.916667,73,-1,-1.0,...,9.328247,5.239314,0.359656,-1,2,5.0,-1.0,12.248477,0.375214,False


In [3]:
def load_X_y(bucket_name, churn_day, is_feature_selection=False, feature_list=[]):
    """ Loads the X & y training & test data from AWS Bucket
    Args: 
        bucket_name (str): The AWS S3 bucket to pull the training and test data from.
        churn_days (int): The number of days since a last order that a user is considered "churned".
        is_feature_selection (bool): Whether or not to use specific features to train the model. Default is False.
        feature_list (list): The names of the features to use if `is_feature_selection` is True. Default is an 
                             empty list.

    Returns:
        X_train (Pandas DataFrame): Predictor values for training dataset
        X_test (Pandas DataFrame): Predictor values for test dataset
        y_train (Pandas Series): Target values for training dataset
        y_test (Pandas Series): Target values for test dataset
    """
    aws_id = os.environ['AWS_ACCESS_KEY_ID']
    aws_secret = os.environ['AWS_SECRET_ACCESS_KEY']
    client = boto3.client('s3'
                          ,aws_access_key_id=aws_id
                          ,aws_secret_access_key=aws_secret)

    train_obj = client.get_object(Bucket=bucket_name, Key=f"churn_{churn_day}_train.csv")
    test_obj = client.get_object(Bucket=bucket_name, Key=f"churn_{churn_day}_test.csv")

    X_train = pd.read_csv(io.BytesIO(train_obj['Body'].read()), encoding='utf8')
    X_train = X_train.drop(['user_id', 'signup_time_utc', 'last_order_time_utc'], axis=1)
    if is_feature_selection:
        X_train = X_train[feature_list]
    y_train = X_train.pop('churned_user')

    X_test = pd.read_csv(io.BytesIO(test_obj['Body'].read()), encoding='utf8')
    X_test = X_test.drop(['user_id', 'signup_time_utc', 'last_order_time_utc'], axis=1)
    if is_feature_selection:
        X_test = X_test[feature_list]
    y_test = X_test.pop('churned_user')

    return X_train, X_test, y_train, y_test

In [4]:
split_data = load_X_y('food-delivery-churn', 30)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
split_data[1]

,city_name,city_group,foreign_user,acquisition_category,acquisition_subcategory,language,signup_to_order_hours,days_since_signup,first_order_driver_rating,first_order_avg_meal_rating,...,first_order_payment,first_order_discount_amount,first_order_discount_percent,first_order_meal_reviews,first_30_day_orders,first_30_day_avg_meal_rating,first_30_day_avg_driver_rating,first_30_day_avg_gmv,first_30_day_discount_percent,first_30_day_subscription_user
0,Tolos,City,True,Brand,DirectMail,domestic,5.583333,49,-1,5.00,...,39.67,8.9900,0.184751,4,3,5.000000,-1.0,57.389333,0.230287,False
1,Gulltown,College Town,True,Referral,Referral,foreign,0.533333,358,5,4.67,...,55.23,5.0000,0.083015,3,3,4.556667,4.5,66.716667,0.039970,False
2,Braavos,City,True,Others,Others,domestic,310.416667,180,5,-1.00,...,82.55,3.9900,0.046106,-1,5,3.000000,5.0,93.753380,0.042552,True
3,Ghozi,City,False,Others,Others,domestic,502.316667,367,-1,5.00,...,21.05,4.9900,0.191628,3,1,5.000000,-1.0,26.040000,0.191628,True
4,Ghozi,City,False,Others,Others,domestic,23.683333,345,-1,-1.00,...,62.52,4.9900,0.073915,-1,1,-1.000000,-1.0,67.510000,0.073915,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35276,Ghozi,City,False,OwnedChannel,InApp,unknown,1581.183333,134,-1,-1.00,...,50.13,3.9923,0.073764,-1,1,-1.000000,-1.0,54.122300,0.073764,False
35277,Pentos,City,True,Others,Others,foreign,0.866667,352,-1,-1.00,...,52.69,0.0000,0.000000,-1,2,-1.000000,-1.0,45.430000,0.055030,False
35278,Tolos,City,True,Others,Others,foreign,299.600000,201,-1,-1.00,...,63.84,2.9940,0.044798,-1,1,-1.000000,-1.0,66.834000,0.044798,False
35279,Qarth,City,True,Partnership,Business&Org,unknown,0.550000,213,-1,-1.00,...,67.26,11.9900,0.151293,-1,3,-1.000000,-1.0,48.366667,0.181048,True
